# Finite Differences and Bessel Functions

Bessel's Equation is 

$$x^2 \frac{d^2 u}{dx^2} + x \frac{d u}{dx} + (x^2 - \alpha^2) u = 0$$

The solutions to this equation are the Bessel function of the first and second kind: $J_\alpha(x)$ and $Y_\alpha(x)$ respectively.  

Many differential equations (esp. PDEs) involve bessel functions.  On the one hand, scipy has these loaded and ready to use.  On the other hand, often times the converge very slowly.  To that end, we will run a couple tests.

1. Examine the convergence of a finite difference scheme on Bessel's equation to a true Bessel function
2. Examine the convergence of a finite difference scheme applied to a PDE which has an analytic solution in terms of Bessel functions.

In a sense this will give some intuition as to when Bessel functions are worth their trouble.

For a controlled experiment, we will look at the following problem:

$$
\nabla^2 u = \frac{\partial u}{\partial t}, \quad x \in \Omega = \left\{x \in \mathbb{R}^2 : r = ||x|| \in \left[0, 1\right]\right\}
$$

Subject to 

$$
\left.\frac{\partial u}{\partial r}\right|_{\partial \Omega} = 0, \quad 
u(r, 0) = 1 - r^4
$$

The solution can be found first by using the symmetry of the domain.  The resulting equation is an PDE in terms of the radius $r$:

$$
\frac{1}{r} \frac{d}{dr} \left(r \frac{d u}{dr}\right) = \frac{\partial u}{\partial t}, \quad 
\frac{\partial u}{\partial r}(r=1, t) = 0, \quad u(r, t=0) = 1 - r^4
$$

This can be solved via separation of variables: let $u(r, t) = R(r) T(t)$. Then

$$
\frac{1}{rR}(r R')' = \frac{T'}{T} = -\lambda^2
$$

Therefore 

$$(rR')' + \lambda^2 r R = 0, \quad R'(1) = 0$$

Which is in Sturm-Liouville form.  Multiplying by $r$ and expanding gives a scaled Bessel Equation

$$
r^2 R'' + rR' + \lambda^2 r^2 R = 0
$$

Letting $z = \lambda r$ gives

$$z^2 R'' + zR' + z^2 R = 0, \quad R'(\lambda) = 0$$

The solution to this equation is $J_0(z)$.  The condition $J_0'(\lambda) = 0$ gives 

$$J_0'(\lambda) = J_{-1}(\lambda) = - J_1(\lambda) = 0 \implies \lambda_n = j_{1, n}$$

Where $j_{1, n}$ is the $n$'th zero of $J_1$.

The solution can be written as:

$$u(r, t) = \sum_{n=1}^\infty \beta_n J_0(j_{1,n} r) e^{-j_{1,n}^2 t}$$

From Sturm-Liouville theory, then we have 

$$
u(r, 0) = 1 - r^4 = \sum_{n=0}^\infty \beta_n J_0(j_{1,n} r), \implies 
\int_0^1 (1 - r^4) J_0(j_{1, n} r) \, r \, dr = \beta_n \int_0^1 J_0^2(j_{1, n} r) \, r \, dr
$$

Which gives the form of $\beta_n$.

At differing values of $t$, the exponential term allows us to use less terms and achieve similar convergences.  
It is known that the maximum value of $J_0$ is 1.  Therefore if we truncate our sum at $n = N$, then the error
is given by

$$\epsilon_N = \beta_{N+1} e^{-j_{1, N+1}^2 t}$$

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import scipy.integrate.quadrature as quad

In [10]:
# zeros of bessel functions
num_zeros = 500
zeros = sp.jn_zeros(1, num_zeros)

f = lambda r: 1 - np.power(r, 4)

# compute βn
β = np.zeros(len(zeros))
for i in range(len(zeros)):
    jn = zeros[i]
    numf = lambda r: f(r) * sp.jv(0, jn * r) * r
    num, nerr = quad(numf, 0, 1)
    denf = lambda r: np.power(sp.jv(0, jn * r), 2) * r
    den, denerr = quad(denf, 0, 1)
    β[i] = num / den
    
print(np.min(np.abs(β)))


0.002743774622588556
